In [8]:
import pandas as pd
import numpy as np
import sep
import glob
import re

In [9]:
from pfs.drp.stella import DetectorMap
from  pfs.utils.dummyCableB import DummyCableBDatabase
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom as geom

In [10]:
from pfs.lam.detAnalysis import removeClosePeak

In [11]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)

In [5]:
%matplotlib widget

In [ ]:
%matplotlib ipympl

In [6]:
afwDisplay.setDefaultBackend("matplotlib")

In [7]:
display = afwDisplay.Display(1)

FigureCanvasNbAgg()

In [12]:
visitId = 23735 #2339#3087 #1059

In [13]:
dmapvisitId = 3451

In [14]:
rerun = "dcb2" # "sm1-march2020"
repo = "sps"
cam = "b2"
arm = cam[0]
sm = int(cam[1])
drpPath = "/drp"

In [15]:
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
calExp = butler.get("calexp", visit=visitId, spectrograph=sm, arm=arm)

In [ ]:
butlerdMap = dafPersist.Butler(f"{drpPath}/{repo}/")
dMap = butlerdMap.get("detectorMap", visit0=dmapvisitId, spectrograph=sm, arm=arm, immediate=True)

In [16]:
rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
rbutler.getKeys('raw')
lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visitId, arm=arm) 
print(lamps)

['Ne']


In [17]:
keywords = ["visit", "spectrograph", "arm"]
for values in rbutler.queryMetadata("raw", keywords, field="OBJECT"):
    dataId = dict(zip(keywords, values))
    print(dataId, rbutler.get("raw", dataId))

In [18]:
#calExp_filetPath = butler.getUri("calexp", visit=visitId, arm="r")

In [19]:
pfsConfig = butler.get("pfsConfig", visit=visitId)
pfsConfig.pfsDesignId

281474976710656

In [ ]:
dcb = DummyCableBDatabase()
fiberSetup = dcb.interpret(pfsConfig.pfsDesignId)
print(fiberSetup)
fiberIds = dcb.getFiberIds(*fiberSetup)
print(fiberIds)

In [ ]:
lines = pd.read_csv("Imqual_LAM_Lines")

In [ ]:
lines

In [ ]:
elements = '|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))
print(elements)
select_lines = lines[(lines.arm == arm) & (lines.element.str.contains(elements))]
print(f"Elements are {select_lines.element.unique()}")

In [ ]:
select_lines

In [ ]:
%matplotlib notebook

In [27]:
selectLines = pd.read_csv("Imqual_LAM_peaklist_2020July.csv")

In [24]:
selectLines =  selectLines[(selectLines.element.str.contains('|'.join(re.findall('[A-Z]µ^A-Z]*', "".join(lamps)))))]
print('|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps))))

Ne


In [25]:
selectLines = selectLines[selectLines.element == "Ne"]

In [29]:
elements = '|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))
print(elements)
selectLines = selectLines[(selectLines.arm == arm) & (selectLines.element.str.contains(elements))]
print(f"Elements are {selectLines.element.unique()}")

Ne
Elements are ['Ne']


In [30]:
selectLines

,fiber,wavelength,element,exptime,arm,X,Y,peak
0,2,585.4110,Ne,2.0,b,4018.7559,3106.6487,1.0
1,63,585.4110,Ne,2.0,b,3641.8413,3092.3406,1.0
2,192,585.4110,Ne,2.0,b,2847.6566,3072.4308,1.0
3,255,585.4110,Ne,2.0,b,2460.1881,3067.5100,1.0
4,339,585.4110,Ne,2.0,b,2012.4911,3065.7355,1.0
...,...,...,...,...,...,...,...,...
75,401,650.8325,Ne,2.0,b,1629.5090,4053.5291,8.0
76,464,650.8325,Ne,2.0,b,1239.6915,4059.6739,8.0
77,525,650.8325,Ne,2.0,b,861.0835,4069.2618,8.0
78,587,650.8325,Ne,2.0,b,474.4227,4083.0019,8.0


In [31]:
display.erase()

In [32]:
# Interpreting displayed mask colors
mask = calExp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

BAD: red
CR: magenta
DETECTED: blue
DETECTED_NEGATIVE: cyan
EDGE: yellow
INTRP: green
NO_DATA: orange
SAT: green
SUSPECT: yellow
UNMASKEDNAN: None


In [33]:
# disable all mask
mask = calExp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    display.setMaskPlaneColor(maskName, "IGNORE")
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

BAD: IGNORE
CR: IGNORE
DETECTED: IGNORE
DETECTED_NEGATIVE: IGNORE
EDGE: IGNORE
INTRP: IGNORE
NO_DATA: IGNORE
SAT: IGNORE
SUSPECT: IGNORE
UNMASKEDNAN: IGNORE


In [34]:
maskPlane = "CR"
calExp.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "magenta")
maskPlane = "SAT"
calExp.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "green")

In [38]:
# display image
display.setMaskTransparency(80)
display.mtv(calExp, title=visitId)

In [36]:
display.scale("linear", "zscale")

In [37]:
with display.Buffering():
    for fiber in fiberIds:
        for wave in select_lines.wavelength.values:
            point = dMap.findPoint(fiber, wave)
            #peaklist.append([fiber, wave, point.getX(), point.getY()])
            display.dot("+",
                   point.getX(),
                   point.getY(),
                   ctype="red")
            display.dot("O",
                   point.getX(),
                   point.getY(),
                   ctype="red")

NameError: name 'fiberIds' is not defined

In [ ]:
with display.Buffering():
    for x,y in selectLines[["X", "Y"]].values:
        display.dot("+",
                   x,
                   y,
                   ctype="green")
        display.dot("O",
                   x,
                   y,
                   ctype="green")

In [ ]:
from pfs.lam.detAnalysis import plotRoiPeak
import matplotlib.pyplot as plt

In [ ]:
selectLines = pd.read_csv("PeakList_SM1_Med_Lines_NIST-2.csv")
selectLines = selectLines[selectLines.element == "Ne"]

In [ ]:
plotRoiPeak(calExp.image.array, selectLines, roi_size=10, scale=True)

In [ ]:
plotRoiLines(calExp.image.array, selectLines, roi_size=10, scale=True)

In [ ]:
def plotRoiLines(image, peak_list, roi_size=20, raw=False, scale=True):
    if type(image) is str:     
        hdulist = fits.open(image, "readonly")
        image = hdulist[1].data
    
    plist = pd.read_csv(peak_list) if type(peak_list) is str else peak_list

    plist = plist.sort_values(["X","Y"], ascending=[True,False])    
    #peak_data = peak_data.reset_index()

    nbfiber = len(plist.fiber.unique())
    nbpeak = len(plist.wavelength.unique())
    # have a list of fiber from 0 to nbfiber 
    listfibertoplot = pd.DataFrame(plist.fiber.unique(), columns=["fiber"])
    
    print("#Fiber= %d and #wavelength= %d"%(nbfiber, nbpeak))
    f, axarr = plt.subplots(nbpeak, nbfiber,  sharex='col', sharey='row',figsize=(12,8))
#    print(axarr.shape)
    vmin=None
    vmax=None
    
    for (wavelength, fiber), group in plist.groupby(['wavelength','fiber']):
        k = int(round(peak))
        i = listfibertoplot[listfibertoplot.fiber == fiber].index.tolist()[0]
#        print(k,i)
        indy = group["X"]
        indx = group["Y"]
        cut_data = image[int(indx-roi_size/2):int(indx+roi_size/2), int(indy-roi_size/2):int(indy+roi_size/2)]
        if nbpeak == 1 and nbfiber == 1:
            axarr.imshow(cut_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
        else:
            axarr[nbpeak -1 -k, nbfiber - i -1].imshow(cut_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
    f.subplots_adjust(hspace=0.5,wspace=0.5)
    plt.show()
def plotRoiPeak(image, peak_list, roi_size=20, raw=False, scale=True):
    if type(image) is str:     
        hdulist = fits.open(image, "readonly")
        image = hdulist[1].data
    
    plist = pd.read_csv(peak_list) if type(peak_list) is str else peak_list

    plist = plist.sort_values(["X","Y"], ascending=[True,False])    
    #peak_data = peak_data.reset_index()

    nbfiber = len(plist.fiber.unique())
    nbpeak = len(plist.peak.unique())
    # have a list of fiber from 0 to nbfiber 
    listfibertoplot = pd.DataFrame(plist.fiber.unique(), columns=["fiber"])
    
    print("#Fiber= %d and #peak= %d"%(nbfiber, nbpeak))
    f, axarr = plt.subplots(nbpeak, nbfiber,  sharex='col', sharey='row',figsize=(12,8))
#    print(axarr.shape)
    vmin=None
    vmax=None
    
    for (peak, fiber), group in plist.groupby(['peak','fiber']):
        k = int(round(peak))
        i = listfibertoplot[listfibertoplot.fiber == fiber].index.tolist()[0]
#        print(k,i)
        indy = group["X"]
        indx = group["Y"]
        cut_data = image[int(indx-roi_size/2):int(indx+roi_size/2), int(indy-roi_size/2):int(indy+roi_size/2)]
        if nbpeak == 1 and nbfiber == 1:
            axarr.imshow(cut_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
        else:
            axarr[nbpeak -1 -k, nbfiber - i -1].imshow(cut_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
    f.subplots_adjust(hspace=0.5,wspace=0.5)
    plt.show()

In [ ]:
# TO be remove or ...

In [ ]:
mask = calExp.mask.array
data = calExp.image.array
threshold = 5
objects = sep.extract(data , threshold, mask=mask) #, filter_kerneµl=None)

In [ ]:
# how many objects were detected
len(objects)

In [ ]:
with display.Buffering():
    for i in range(len(objects)):
        display.dot("+",
                   objects['x'][i],
                   objects['y'][i],
                   ctype="yellow")

In [ ]:
df = pd.DataFrame(objects, columns=objects.dtype.names)

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'x': 'px','y': 'py', 'peak': 'brightness'})

In [ ]:
df["fiber"]= df.apply(lambda x: dMap.findFiberId(geom.Point2D(float(x["px"]), float(x["py"]))), axis=1)

In [ ]:
df["wavelength"]= df.apply(lambda x: dMap.getWavelength(int(x["fiber"]), float(x["py"])), axis=1)

In [ ]:
fiber = 2

In [ ]:
ax = df[df.fiber == fiber].plot.scatter(x="wavelength", y="brightness", logy=True)
ax2 = ax.twiny()
df[df.fiber == fiber].plot.scatter(x="py", y="brightness", ax=ax2)
ax.hlines(2000, df.wavelength.min(), df.wavelength.max(), colors="r")

In [ ]:
dist = 40 # 80 for thFocus, can be decrease for imquality
dfp = removeClosePeak(df, dist=dist ,doPlot=True)

In [ ]:
dfp = df[(dfp.fiber == fiber) & (df.brightness > 2000)]

In [ ]:
with display.Buffering():
    for x,y in dfp[["px", "py"]].values:
        display.dot("+",
                   x,
                   y,
                   ctype="green")
        display.dot("O",
                   x,
                   y,
                   ctype="green")

In [ ]:
display.erase()

In [ ]:
exptime = rbutler.queryMetadata('raw', ['exptime'], visit=visitId, arm=arm) 
exptime = np.round(exptime[0])

wavelength element outfocus status

In [ ]:
selectedLines = dfp[["wavelength","brightness"]]

In [ ]:
selectedLines["element"] = lamps[0]
selectedLines["outfocus"] = 0
selectedLines["status"] = 1
selectedLines["exptime"] = exptime

In [ ]:
selectedLines

In [ ]:
selectedLines.sort_values("wavelength").to_csv("Imqual_Lines_FMa_Med_HgAr.txt", index=False)

then we need manual checking with NIST 

In [ ]:
tmp = pd.read_csv("Imqual_Lines_FMa_Med_HgAr.txt")

In [ ]:
tmp

In [ ]:
tmp = tmp.to_csv("Imqual_Lines_FMa_Med_HgAr.txt", index=False)

In [ ]:
nist_lines = pd.read_csv("Imqual_Lines_FMa_Blue_NIST.txt")

In [ ]:
nist_lines

In [ ]:
peaklist = []
for fiber in fiberIds:
    for wave in nist_lines.wavelength.values:
        point = dMap.findPoint(fiber, wave)
        peaklist.append([fiber, wave, point.getX(), point.getY()])

In [ ]:
peaksList = pd.DataFrame(peaklist, columns=['fiber', 'wavelength', 'X', 'Y'])

In [ ]:
for (fiber),a in peaksList.sort_values("wavelength").groupby(["fiber"]):
    l = len(peaksList[(peaksList.fiber == fiber)])
#    print(l)
    peaksList.loc[peaksList.fiber == fiber, "peak"] = np.arange(1,l+1)

In [ ]:
peaksList.head()

In [ ]:
peaksList = peaksList.merge(nist_lines, on="wavelength")

In [ ]:
peaksList.head()

In [ ]:
peaksList.to_csv("PeakList_SM1_Blue_Lines_NIST.csv", index=False)

In [ ]:
peaks = peaksList[["X","Y"]].values

In [ ]:
with display.Buffering():
    for x,y in peaks:
        display.dot("+",
                   x,
                   y,
                   ctype="orange")
        display.dot("O",
                   x,
                   y,
                   ctype="orange")

In [ ]:
tmp =  peaksList

In [ ]:
searchFile = f"Imqual_Lines_FMa_Blue_*_NIST.txt"
print(searchFile)

filelist = glob.glob(searchFile)
print('\n'.join(filelist))

In [ ]:
ob = pd.concat([pd.read_csv(filen) for filen in filelist ], ignore_index=True)

In [ ]:
ob

In [ ]:
ob.to_csv("Imqual_Lines_FMa_Blue_NIST.txt", index=False)

In [ ]:
selectLines = pd.read_csv("usedLines_HgAr_630-970.csv")
selectLines = pd.read_csv("usedLines_Ne_630-970.csv")

In [ ]:
selectLines[["wavelength", "element"]]

In [ ]:
with display.Buffering():
    for fiber in fiberIds:
        for wave in selectLines.wavelength.values:
            point = dmap.findPoint(fiber, wave)
            #peaklist.append([fiber, wave, point.getX(), point.getY()])
            display.dot("+",
                   point.getX(),
                   point.getY(),
                   ctype="red")
            display.dot("O",
                   point.getX(),
                   point.getY(),
                   ctype="red")

In [ ]:
peaks = pd.read_csv("SM1_b1_A_hgar_Exp1342_Nov2019.csv")

In [ ]:
peaks

In [ ]:
peaks["wavelength"]= peaks.apply(lambda x: dmap.getWavelength(int(x["fiber"]), float(x["Y"])), axis=1)

In [ ]:
with display.Buffering():
    for fiber in peaks.fiber.values:
        for wave in peaks.wavelength.values:
            point = dmap.findPoint(fiber, wave)
            #peaklist.append([fiber, wave, point.getX(), point.getY()])
            display.dot("+",
                   point.getX(),
                   point.getY(),
                   ctype="green")
            display.dot("O",
                   point.getX(),
                   point.getY(),
                   ctype="green")

In [ ]:
dmap.getWavelength(339, 85.2)

In [ ]:
broadLineListFilename = "broadLines.txt"
broadLines = pd.read_csv(broadLineListFilename, comment="#", delim_whitespace=True, 
                    names=['wavelength', 'air', 'element'])

In [ ]:
with display.Buffering():
    for fiber in fiberIds:
        for wave in broadLines.wavelength.values:
            point = dmap.findPoint(fiber, wave)
            #peaklist.append([fiber, wave, point.getX(), point.getY()])
            display.dot("+",
                   point.getX(),
                   point.getY(),
                   ctype="red")
            display.dot("O",
                   point.getX(),
                   point.getY(),
                   ctype="red")